In [1]:
import os
import time
import copy
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image

In [22]:
path = 'data/'
df = pd.DataFrame(pd.read_csv(path+'data.csv'))
df = df[df['PhotoType']=='ДС']
df = df.loc[:,['Folder','Id','Ruin']]
df = df[df['Ruin']!='частично разрушен']
df.head(10)

,Folder,Id,Ruin
0,Unload1,1000000,не разрушен
2,Unload1,1000002,не разрушен
4,Unload1,1000004,не разрушен
6,Unload1,1000006,не разрушен
8,Unload1,1000008,не разрушен
10,Unload1,1000010,не разрушен
12,Unload1,1000012,не разрушен
14,Unload1,1000014,не разрушен
16,Unload1,1000016,не разрушен
18,Unload1,1000018,не разрушен


In [23]:
print(df[df['Ruin']=='разрушен'].count())
print(df[df['Ruin']=='частично разрушен'].count())
print(df[df['Ruin']=='не разрушен'].count())

Folder    169
Id        169
Ruin      169
dtype: int64
Folder    0
Id        0
Ruin      0
dtype: int64
Folder    1472
Id        1472
Ruin      1472
dtype: int64


In [29]:
nor_indices = []

for row in df.itertuples():
    if row[3] == 'не разрушен':
        nor_indices.append(row[0])    
drop_indices = np.random.choice(nor_indices, 
                                1303, 
                                replace=False)

df  = df.drop(drop_indices)

In [40]:
X = []
y = []

transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

i = 1
for row in df.itertuples():
    img = Image.open(path+row[1]+'/data/'+str(row[2])+'.jpeg')
    w, h = img.size
    area = [w / 2 - 112, 5, w / 2 + 112, 229]
    while h >= area[3] + 5:
        cropped_img = img.crop(area)
        X.append(transform(cropped_img))
        if row[3] == 'не разрушен':
            l = 0
        else:
            l = 1
        y.append(l)
        area[1] += 224
        area[3] += 224
        
    if i % 50 == 0 or i == df.shape[0]:
        print(str(i)+'/'+str(df.shape[0]))
    i += 1 

50/338
100/338
150/338
200/338
250/338
300/338
338/338


In [41]:
print(len(X))

6712


In [42]:
torch.manual_seed(0)
np.random.seed(0)

In [43]:
feature_extract = True
use_pretrained = True
num_classes = 2

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
model_ft = models.squeezenet1_0(pretrained=use_pretrained)
set_parameter_requires_grad(model_ft, feature_extract)
model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model_ft.num_classes = num_classes
input_size = 224

In [46]:
device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"
model_ft = model_ft.to(device)
params_to_update = model_ft.parameters()
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [47]:
data = [[X[i], y[i]] for i in range(len(X))]
test_indices = np.random.choice(np.arange(len(data)), 400, replace=False)
data_test = [data[i] for i in test_indices]
data = [data[i] for i in range(len(data)) if i not in test_indices]
val_indices = np.random.choice(np.arange(len(data)), 400, replace=False)
data_val = [data[i] for i in val_indices]
data_train = [data[i] for i in range(len(data)) if i not in val_indices]
data = {'train': data_train, 'val': data_val, 'test': data_test}
batch_size = 8
dataloaders_dict = {x: torch.utils.data.DataLoader(data[x], batch_size=batch_size, shuffle=True, num_workers=4) 
                    for x in ['train', 'val']}

In [48]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval() 

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history 

In [49]:
num_epochs = 15
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/14
----------
train Loss: 0.1623 Acc: 0.9411
val Loss: 0.1130 Acc: 0.9550

Epoch 1/14
----------
train Loss: 0.1238 Acc: 0.9579
val Loss: 0.1417 Acc: 0.9550

Epoch 2/14
----------
train Loss: 0.1146 Acc: 0.9601
val Loss: 0.0990 Acc: 0.9550

Epoch 3/14
----------
train Loss: 0.1044 Acc: 0.9645
val Loss: 0.1043 Acc: 0.9625

Epoch 4/14
----------
train Loss: 0.1051 Acc: 0.9631
val Loss: 0.0907 Acc: 0.9625

Epoch 5/14
----------
train Loss: 0.1027 Acc: 0.9685
val Loss: 0.0977 Acc: 0.9500

Epoch 6/14
----------
train Loss: 0.1000 Acc: 0.9655
val Loss: 0.1278 Acc: 0.9625

Epoch 7/14
----------
train Loss: 0.0995 Acc: 0.9657
val Loss: 0.0914 Acc: 0.9600

Epoch 8/14
----------
train Loss: 0.0971 Acc: 0.9672
val Loss: 0.0996 Acc: 0.9600

Epoch 9/14
----------
train Loss: 0.0951 Acc: 0.9660
val Loss: 0.0959 Acc: 0.9600

Epoch 10/14
----------
train Loss: 0.0960 Acc: 0.9685
val Loss: 0.0843 Acc: 0.9675

Epoch 11/14
----------
train Loss: 0.0919 Acc: 0.9696
val Loss: 0.0857 Acc: 0.9575

Ep

In [50]:
testloader = torch.utils.data.DataLoader(data['test'], batch_size=batch_size, shuffle=False, num_workers=4) 

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 97 %
